In [1]:
# Imports & IST timezone
import pandas as pd
import numpy as np
from datetime import datetime, time
import plotly.graph_objects as go

try:
    from zoneinfo import ZoneInfo
    IST = ZoneInfo("Asia/Kolkata")
except:
    IST = None


In [2]:
fp = "Downloads/Play Store Data.csv"
ps = pd.read_csv(fp)
print("Loaded rows:", ps.shape[0], "cols:", ps.shape[1])
ps.head(3)


Loaded rows: 10841 cols: 13


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


In [3]:
ps.index=range(1,len(ps)+1)

In [4]:
ps.head(3)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
3,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


In [5]:
def parse_installs(x):
    if pd.isna(x): return np.nan
    return pd.to_numeric(str(x).replace('+','').replace(',','').strip(), errors='coerce')

def parse_size(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    if 'Varies' in s or s=='':
        return np.nan
    if s.lower().endswith('m'):
        return pd.to_numeric(s[:-1], errors='coerce') * 1_000_000
    if s.lower().endswith('k'):
        return pd.to_numeric(s[:-1], errors='coerce') * 1_000
    return pd.to_numeric(s, errors='coerce')

def parse_price(x):
    if pd.isna(x): return 0.0
    s = str(x)
    # remove currency symbols and commas
    s = s.replace('$','').replace('₹','').replace(',','').strip()
    # sometimes 'Free' or '0' present
    if s.lower() in ('free','0','0.0',''):
        return 0.0
    return pd.to_numeric(s, errors='coerce')

def parse_android(x):
    if pd.isna(x): return np.nan
    s = str(x)
    # take first numeric part e.g. '4.1 and up' -> 4.1
    import re
    m = re.search(r'(\d+(\.\d+)?)', s)
    if m:
        return float(m.group(1))
    return np.nan

In [6]:
colmap = {
    'App':'app',
    'Category':'category',
    'Installs':'installs',
    'Price':'price',
    'Size':'size',
    'Android Ver':'android_ver',
    'Last Updated':'last_updated',
    'Content Rating':'content_rating',
    'Type':'type'  
    # Free / Paid
}
for old,new in colmap.items():
    if old in ps.columns:
        ps.rename(columns={old:new}, inplace=True)

# Parse fields
if 'installs' in ps.columns:
    ps['installs_n'] = ps['installs'].apply(parse_installs)
if 'size' in ps.columns:
    ps['size_b'] = ps['size'].apply(parse_size)
if 'price' in ps.columns:
    ps['price_usd'] = ps['price'].apply(parse_price)
if 'android_ver' in ps.columns:
    ps['android_num'] = ps['android_ver'].apply(parse_android)
    
ps['app'] = ps['app'].astype('string') if 'app' in ps.columns else ps.get('App', pd.Series([], dtype='string'))
ps['category'] = ps['category'].astype('string') if 'category' in ps.columns else ps.get('Category', pd.Series([], dtype='string'))
ps['type'] = ps['type'].astype('string') if 'type' in ps.columns else ps.get('Type', pd.Series([], dtype='string'))

print("Parsed columns present:", [c for c in ['installs_n','size_b','price_usd','android_num'] if c in ps.columns])
ps.head(2)


Parsed columns present: ['installs_n', 'size_b', 'price_usd', 'android_num']


,app,category,Rating,Reviews,size,installs,type,price,content_rating,Genres,last_updated,Current Ver,android_ver,installs_n,size_b,price_usd,android_num
1,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,10000.0,19000000.0,0.0,4.0
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,500000.0,14000000.0,0.0,4.0


In [7]:
# Apply exact filters from Task instructions
df = ps.copy()

# 1) installs >= 10,000
if 'installs_n' in df.columns:
    df = df[df['installs_n'] >= 10_000]
else:
    raise ValueError("Column 'Installs' missing after parse.")

#  revenue >= 10,000 -> compute revenue proxy as price * installs (dataset likely lacks direct revenue)
# revenue proxy used only if 'price_usd' present; else raise note.
if 'price_usd' in df.columns and 'installs_n' in df.columns:
    df['revenue_proxy'] = (df['price_usd'].fillna(0.0)) * df['installs_n'].fillna(0)
else:
    df['revenue_proxy'] = 0.0

df = df[df['revenue_proxy'] >= 10_000]

# android version > 4.0
if 'android_num' in df.columns:
    df = df[df['android_num'] > 4.0]
else:
    # if column not present, try common alternate columns
    raise ValueError("Required Android version column missing or unparsable.")

# size > 15MB
if 'size_b' in df.columns:
    df = df[df['size_b'] > 15_000_000]
else:
    raise ValueError("Required Size column missing or unparsable.")

#  content rating should be 'Everyone'
if 'content_rating' in df.columns:
    df['content_rating'] = df['content_rating'].astype('string').str.strip()
    df = df[df['content_rating'].str.lower() == 'everyone']
else:
    raise ValueError("Required Content Rating column missing.")

#  app name length <= 30 chars (including spaces and special chars)
df['app_len'] = df['app'].astype('string').apply(lambda x: len(str(x)))
df = df[df['app_len'] <= 30]

print("After filters rows:", df.shape[0])
df[['app','category','type','installs_n','price_usd','revenue_proxy','android_num','size_b','content_rating','app_len']].head(3)


After filters rows: 33


,app,category,type,installs_n,price_usd,revenue_proxy,android_num,size_b,content_rating,app_len
854,Toca Life: City,EDUCATION,Paid,500000.0,3.99,1995000.0,4.4,24000000.0,Everyone,15
855,Toca Life: Hospital,EDUCATION,Paid,100000.0,3.99,399000.0,4.4,24000000.0,Everyone,19
1336,Meditation Studio,HEALTH_AND_FITNESS,Paid,10000.0,3.99,39900.0,4.3,29000000.0,Everyone,17


In [8]:
# Top 3 categories by total installs, then aggregate free vs paid ===
if 'category' not in df.columns:
    raise ValueError("Category column missing.")

cat_tot = df.groupby('category', as_index=False).agg(total_installs=('installs_n','sum'))
top3 = cat_tot.sort_values('total_installs', ascending=False).head(3)['category'].tolist()

print("Top 3 categories selected:", top3)

df_top3 = df[df['category'].isin(top3)].copy()


agg = (df_top3.groupby(['category','type'], as_index=False)
         .agg(avg_installs=('installs_n','mean'),
              avg_revenue=('revenue_proxy','mean'),
              count_apps=('app','nunique')))

# Pivot so free vs paid become rows for plotting ease
agg


Top 3 categories selected: ['PHOTOGRAPHY', 'FAMILY', 'PERSONALIZATION']


,category,type,avg_installs,avg_revenue,count_apps
0,FAMILY,Paid,381428.571429,7.969000e+05,7
1,PERSONALIZATION,Paid,670000.000000,6.666333e+05,2
2,PHOTOGRAPHY,Paid,1000000.000000,5.990000e+06,1


In [9]:
agg.index=range(1,len(agg)+1)

In [10]:
agg

,category,type,avg_installs,avg_revenue,count_apps
1,FAMILY,Paid,381428.571429,7.969000e+05,7
2,PERSONALIZATION,Paid,670000.000000,6.666333e+05,2
3,PHOTOGRAPHY,Paid,1000000.000000,5.990000e+06,1


In [11]:
# Time gate (1 PM – 2 PM IST) and Dual-axis chart
def allowed_time():
    now_t = datetime.now(IST).time() if IST else datetime.now().time()
    return time(13,0) <= now_t <= time(14,0)

if agg.empty:
    print("No data available after applying filters — please check the filters.")
else:
    if allowed_time():
        # Prepare plot per category: for each category show two bars (Free, Paid) for avg_installs,
        # and two lines or secondary bars for avg_revenue.
        # We'll create one figure with secondary y-axis.
        from plotly.subplots import make_subplots

        cats = top3
        types = ['Free','Paid']
        # create mapping for easier plotting
        # get avg_installs matrix [category x type]
        inst_matrix = { (r['category'], r['type']): r['avg_installs'] for _,r in agg.iterrows() }
        rev_matrix = { (r['category'], r['type']): r['avg_revenue'] for _,r in agg.iterrows() }

        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # X positions: we'll use category + type labels
        x_labels = []
        installs_vals = []
        revenue_vals = []
        color_map = {'Free':'#1f77b4', 'Paid':'#ff7f0e'}

        for cat in cats:
            for t in types:
                x_labels.append(f"{cat}\n({t})")
                installs_vals.append(inst_matrix.get((cat,t), 0))
                revenue_vals.append(rev_matrix.get((cat,t), 0))

        # Bars for avg installs     (primary y)
        fig.add_trace(
            go.Bar(x=x_labels, y=installs_vals, name='Avg Installs', marker_color='steelblue'),
            secondary_y=False
        )

        # Line (or bar) for avg revenue on secondary y
        fig.add_trace(
            go.Scatter(x=x_labels, y=revenue_vals, mode='lines+markers+text', name='Avg Revenue (USD)',
                       text=[f"{int(v):,}" for v in revenue_vals], textposition='top center'),
            secondary_y=True
        )

        fig.update_layout(
            title_text="Avg Installs vs Avg Revenue (Free vs Paid) — Top 3 Categories (Filtered)",
            xaxis_tickangle=-45,
            barmode='group',
            legend=dict(orientation='h', y=1.12),
            margin=dict(t=80, b=160)
        )

        fig.update_yaxes(title_text="Average Installs", secondary_y=False)
        fig.update_yaxes(title_text="Average Revenue (USD)", secondary_y=True)

        fig.show()
    else:
        print("This graph will be visible on the dashboard only between 1:00 PM and 2:00 PM IST. (It is disabled at the current time.)")


This graph will be visible on the dashboard only between 1:00 PM and 2:00 PM IST. (It is disabled at the current time.)


In [12]:
#  Optional save HTML (run when plot visible)
try:
    fig.write_html("Task3_DualAxis.html")
    print("Saved Task3_DualAxis.html")
except NameError:
    print("Note: Plot not created (time window outside or agg empty), so no HTML saved.")


Note: Plot not created (time window outside or agg empty), so no HTML saved.
